<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [4]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
# Define launch sites with coordinates
launch_sites = [
    {"name": "NASA Johnson Space Center", "coords": [29.559684888503615, -95.0830971930759]},
    {"name": "CCAFS LC-40", "coords": [28.562302, -80.577356]},
    {"name": "VAFB SLC-4E", "coords": [34.632834, -120.610746]},
    {"name": "KSC LC-39A", "coords": [28.573255, -80.646895]}
]

# Create a map centered on the first site
site_map = folium.Map(location=launch_sites[0]["coords"], zoom_start=5)

# Add circles and markers for each launch site
for site in launch_sites:
    folium.Circle(
        location=site["coords"],
        radius=1000,
        color='#d35400',
        fill=True,
        fill_color='#d35400',
        fill_opacity=0.4
    ).add_child(folium.Popup(site["name"])).add_to(site_map)

    folium.Marker(
        location=site["coords"],
        icon=DivIcon(
            icon_size=(150,36),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site["name"]}</b></div>'
        )
    ).add_to(site_map)



# Add the equator line as a PolyLine across longitude
equator_line = folium.PolyLine(
    locations=[[0, -180], [0, 180]],
    color='red',
    weight=2,
    opacity=0.7,
    dash_array='5, 5'
)

# Save the map
site_map.save("launch_sites_with_equator.html")

# Display
site_map.add_child(circle)
site_map.add_child(marker)
site_map.add_child(equator_line)

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


#### Are all launch sites in proximity to the Equator line?
* None of the launch sites is in proximity to the equator
#### Are all launch sites in very close proximity to the coast?
* Yes, VABF SLC-4E is on the coast of California to the pacific ocean, while CCAFS LC-40 is on the coast of Florida to the Atlantic ocean

In [10]:
# Task 2: Mark the success/failed launches for each site on the map

launch_data = [
    {"site": "NASA Johnson Space Center", "coords": [29.559684888503615, -95.0830971930759], "outcome": "success"},
    {"site": "CCAFS LC-40", "coords": [28.562302, -80.577356], "outcome": "failure"},
    {"site": "VAFB SLC-4E", "coords": [34.632834, -120.610746], "outcome": "success"},
    {"site": "KSC LC-39A", "coords": [28.573255, -80.646895], "outcome": "failure"},
    {"site": "KSC LC-39A", "coords": [28.573255, -80.646895], "outcome": "success"}  # Multiple launches from same site
]

for launch in launch_data:
    color = 'green' if launch["outcome"] == "success" else 'red'
    folium.Marker(
        location=launch["coords"],
        popup=f'{launch["site"]} - {launch["outcome"].capitalize()}',
        icon=folium.Icon(color=color, icon='rocket', prefix='fa')
    ).add_to(site_map)

site_map.save("launch_sites_with_outcomes.html")

# Display the map
display(site_map)

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


In [50]:
# Check if 'Launch Site' is in the DataFrame
if 'Launch Site' in spacex_df.columns:
    print("Feature exists!")
else:
    print("Feature not found.")

# Find rows where Launch Site is 'CCAFS LC-40'
filtered_df = spacex_df[spacex_df['Launch Site'] == 'CCAFS LC-40']
print(filtered_df.head())

# List all features (columns)
print(spacex_df.columns.tolist())



Feature exists!
   Launch Site        Lat       Long  class marker_color  distance_to_site_km
0  CCAFS LC-40  28.562302 -80.577356      0          red             0.112472
1  CCAFS LC-40  28.562302 -80.577356      0          red             0.112472
2  CCAFS LC-40  28.562302 -80.577356      0          red             0.112472
3  CCAFS LC-40  28.562302 -80.577356      0          red             0.112472
4  CCAFS LC-40  28.562302 -80.577356      0          red             0.112472
['Launch Site', 'Lat', 'Long', 'class', 'marker_color', 'distance_to_site_km']


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [12]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [13]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red

# Create a new column 'marker_color' based on the 'class' column
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [14]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

In [15]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Assuming spacex_df already exists and includes 'Launch Site', 'Lat', 'Long', and 'class'

# Step 1: Add marker_color column based on class
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Step 2: Create a map centered around the average coordinates
avg_lat = spacex_df['Lat'].mean()
avg_long = spacex_df['Long'].mean()
site_map = folium.Map(location=[avg_lat, avg_long], zoom_start=6)

# Step 3: Create a MarkerCluster and add it to the map
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)

# Step 4: Loop through all rows and add markers
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'], icon='rocket', prefix='fa'),
        popup=f"<b>Launch Site:</b> {record['Launch Site']}<br><b>Outcome:</b> {'Success' if record['class'] == 1 else 'Failure'}"
    )
    marker_cluster.add_child(marker)

# Step 5: Display the map
site_map


In [16]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Sample DataFrame (replace with your full spacex_df if already loaded)
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Create the map centered around average coordinates
avg_lat = spacex_df['Lat'].mean()
avg_long = spacex_df['Long'].mean()
site_map = folium.Map(location=[avg_lat, avg_long], zoom_start=6)

# Create a MarkerCluster
marker_cluster = MarkerCluster()
site_map.add_child(marker_cluster)

# Add markers with site name labels
for index, record in spacex_df.iterrows():
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'], icon='rocket', prefix='fa'),
        popup=folium.Popup(
            f"<b>Launch Site:</b> {record['Launch Site']}<br><b>Outcome:</b> {'Success' if record['class'] == 1 else 'Failure'}",
            max_width=250
        ),
        tooltip=record['Launch Site']  # This shows the site name on hover
    )
    marker_cluster.add_child(marker)

# Display the map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [42]:
# TASK 3: Calculate the distances between a launch site to its proximities

print(spacex_df['Launch Site'].unique())

import pandas as pd
from math import radians, cos, sin, asin, sqrt

# Step 1: Define launch site coordinates
site_coords = {
    "KSC LC-39A": (28.573255, -80.646895),
    "CCAFS SLC-40": (28.563197, -80.576820),
    "CCAFS LC-40": (28.563197, -80.576820),
    "VAFB SLC-4E": (34.632834, -120.610746),
    "NASA Johnson Space Center": (29.559684888503615, -95.0830971930759)
}

# Step 2: Clean and standardize launch site names
def standardize_site_name(name):
    name = name.strip()
    alias_map = {
        "CCAFS LC-40": "CCAFS SLC-40",  # common typo
        "JSC": "NASA Johnson Space Center"
    }
    return alias_map.get(name, name)

# Step 3: Haversine formula to calculate distance in kilometers
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Radius of Earth in kilometers
    return c * r

# Step 4: Apply distance calculation safely
def compute_distance(row):
    site = standardize_site_name(row['Launch Site'])
    if site in site_coords:
        site_lat, site_lon = site_coords[site]
        return haversine(site_lat, site_lon, row['Lat'], row['Long'])
    else:
        return None  # or np.nan if preferred

# Step 5: Calculate distances
spacex_df['distance_to_site_km'] = spacex_df.apply(compute_distance, axis=1)

# Step 6: Preview results
print(spacex_df[['Launch Site', 'Lat', 'Long', 'distance_to_site_km']].head())


['CCAFS LC-40' 'VAFB SLC-4E' 'KSC LC-39A' 'CCAFS SLC-40']
   Launch Site        Lat       Long  distance_to_site_km
0  CCAFS LC-40  28.562302 -80.577356             0.112472
1  CCAFS LC-40  28.562302 -80.577356             0.112472
2  CCAFS LC-40  28.562302 -80.577356             0.112472
3  CCAFS LC-40  28.562302 -80.577356             0.112472
4  CCAFS LC-40  28.562302 -80.577356             0.112472


In [43]:
import pandas as pd
from math import radians, cos, sin, asin, sqrt

# Step 1: Define launch site reference coordinates
site_coords = {
    "KSC LC-39A": (28.573255, -80.646895),
    "CCAFS SLC-40": (28.563197, -80.576820),
    "CCAFS LC-40": (28.563197, -80.576820),
    "VAFB SLC-4E": (34.632834, -120.610746),
    "NASA Johnson Space Center": (29.559684888503615, -95.0830971930759)
}

# Step 2: Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Earth radius in kilometers
    return c * r

# Step 3: Get unique site coordinates from your DataFrame
unique_sites_df = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates()

# Step 4: Calculate distance to reference site center
unique_sites_df['distance_to_site_km'] = unique_sites_df.apply(
    lambda row: haversine(
        site_coords[row['Launch Site']][0],
        site_coords[row['Launch Site']][1],
        row['Lat'],
        row['Long']
    ),
    axis=1
)

# Step 5: Preview results
print(unique_sites_df)


     Launch Site        Lat        Long  distance_to_site_km
0    CCAFS LC-40  28.562302  -80.577356             0.112472
26   VAFB SLC-4E  34.632834 -120.610745             0.000049
36    KSC LC-39A  28.573255  -80.646895             0.000056
49  CCAFS SLC-40  28.563197  -80.576820             0.000020


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [19]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [20]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [21]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [35]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [40]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)

# Step 1: Define coordinates as a list of [lat, lon] pairs
coordinates = [
    [launch_site_lat, launch_site_lon],
    [coastline_lat, coastline_lon]
]

# Step 2: Create the PolyLine object
lines = folium.PolyLine(
    locations=coordinates,
    weight=1,
    color='purple',
    tooltip='Launch Site to Coastline'
)

# Step 3: Add the line to the map
site_map.add_child(lines)


In [23]:
import folium
from folium.plugins import MousePosition
from folium.features import DivIcon
from math import radians, cos, sin, asin, sqrt

# Step 1: Define launch site coordinates
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Step 2: Define manually selected coastline coordinates (from MousePosition)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Step 3: Haversine distance function
def calculate_distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Earth radius in kilometers
    return c * r

# Step 4: Calculate distance
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Step 5: Create folium map centered on launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Step 6: Add MousePosition plugin
mouse_position = MousePosition(
    position='topright',
    separator=' | ',
    prefix='Coastline Coordinates:',
    lat_formatter="function(num) {return L.Util.formatNum(num, 6);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 6);}"
)
site_map.add_child(mouse_position)

# Step 7: Add marker for launch site
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup='Launch Site',
    icon=folium.Icon(color='green')
).add_to(site_map)

# Step 8: Add marker for coastline point
folium.Marker(
    location=[coastline_lat, coastline_lon],
    popup='Closest Coastline',
    icon=folium.Icon(color='blue')
).add_to(site_map)

# Step 9: Add distance label using DivIcon
distance_marker = folium.Marker(
    location=[(launch_site_lat + coastline_lat)/2, (launch_site_lon + coastline_lon)/2],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_coastline),
    )
)
distance_marker.add_to(site_map)

# Step 10: Draw a line between launch site and coastline
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color='orange',
    weight=2.5,
    opacity=0.8
).add_to(site_map)

# Display the map
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [27]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [31]:
import folium
from folium.features import DivIcon
from math import radians, cos, sin, asin, sqrt

# --- Launch Site Coordinates (e.g., KSC LC-39A) ---
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# --- Closest Railway Coordinates (e.g., Florida East Coast Railway near Titusville) ---
railway_lat = 28.6100
railway_lon = -80.8070

# --- Haversine Distance Function ---
def calculate_distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Earth radius in kilometers
    return c * r

# --- Calculate Distance ---
distance_km = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

# --- Create Map Centered on Launch Site ---
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# --- Add Launch Site Marker ---
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup='Launch Site',
    icon=folium.Icon(color='green')
).add_to(site_map)

# --- Add Railway Marker ---
folium.Marker(
    location=[railway_lat, railway_lon],
    popup='Closest Railway',
    icon=folium.Icon(color='red', icon='train', prefix='fa')
).add_to(site_map)

# --- Draw Line Between Launch Site and Railway ---
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [railway_lat, railway_lon]],
    color='red',
    weight=2,
    tooltip='Distance to Railway'
).add_to(site_map)

# --- Add Distance Label ---
folium.Marker(
    location=[(launch_site_lat + railway_lat)/2, (launch_site_lon + railway_lon)/2],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#c0392b;"><b>{:.2f} KM</b></div>'.format(distance_km),
    )
).add_to(site_map)

# --- Display Map ---
site_map


In [29]:
import folium
from folium.plugins import MousePosition
from folium.features import DivIcon
from math import radians, cos, sin, asin, sqrt

# --- Launch Site Coordinates (example: KSC LC-39A) ---
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# --- Closest Highway Coordinates (manually selected using MousePosition) ---
highway_lat = 28.56445     # Example: highway near Cape Canaveral
highway_lon = -80.57085

# --- Haversine Distance Function ---
def calculate_distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Earth radius in kilometers
    return c * r

# --- Calculate Distance ---
distance_km = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

# --- Create Map Centered on Launch Site ---
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# --- Add MousePosition Plugin ---
mouse_position = MousePosition(
    position='topright',
    separator=' | ',
    prefix='Highway Coordinates:',
    lat_formatter="function(num) {return L.Util.formatNum(num, 6);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 6);}"
)
site_map.add_child(mouse_position)

# --- Add Launch Site Marker ---
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup='Launch Site',
    icon=folium.Icon(color='green')
).add_to(site_map)

# --- Add Highway Marker ---
folium.Marker(
    location=[highway_lat, highway_lon],
    popup='Closest Highway',
    icon=folium.Icon(color='gray', icon='road', prefix='fa')
).add_to(site_map)

# --- Draw Line Between Launch Site and Highway ---
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]],
    color='black',
    weight=2,
    tooltip='Distance to Highway'
).add_to(site_map)

# --- Add Distance Label ---
folium.Marker(
    location=[(launch_site_lat + highway_lat)/2, (launch_site_lon + highway_lon)/2],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#2c3e50;"><b>{:.2f} KM</b></div>'.format(distance_km),
    )
).add_to(site_map)

# --- Display Map ---
site_map


In [30]:
import folium
from folium.features import DivIcon
from math import radians, cos, sin, asin, sqrt

# --- Launch Site Coordinates (e.g., KSC LC-39A) ---
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# --- Closest City Coordinates (e.g., Titusville, FL) ---
city_lat = 28.6122
city_lon = -80.8076

# --- Haversine Distance Function ---
def calculate_distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Earth radius in kilometers
    return c * r

# --- Calculate Distance ---
distance_km = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# --- Create Map Centered on Launch Site ---
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# --- Add Launch Site Marker ---
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup='Launch Site',
    icon=folium.Icon(color='green')
).add_to(site_map)

# --- Add Closest City Marker ---
folium.Marker(
    location=[city_lat, city_lon],
    popup='Closest City: Titusville',
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(site_map)

# --- Draw Line Between Launch Site and City ---
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [city_lat, city_lon]],
    color='blue',
    weight=2,
    tooltip='Distance to City'
).add_to(site_map)

# --- Add Distance Label ---
folium.Marker(
    location=[(launch_site_lat + city_lat)/2, (launch_site_lon + city_lon)/2],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#34495e;"><b>{:.2f} KM</b></div>'.format(distance_km),
    )
).add_to(site_map)

# --- Display Map ---
site_map


In [32]:
import folium
from folium.features import DivIcon
from math import radians, cos, sin, asin, sqrt

# --- Launch Site Coordinates (e.g., KSC LC-39A) ---
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# --- Closest Airport Coordinates (e.g., Space Coast Regional Airport - KTIX) ---
airport_lat = 28.5146
airport_lon = -80.7990

# --- Haversine Distance Function ---
def calculate_distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Earth radius in kilometers
    return c * r

# --- Calculate Distance ---
distance_km = calculate_distance(launch_site_lat, launch_site_lon, airport_lat, airport_lon)

# --- Create Map Centered on Launch Site ---
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# --- Add Launch Site Marker ---
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup='Launch Site',
    icon=folium.Icon(color='green')
).add_to(site_map)

# --- Add Airport Marker ---
folium.Marker(
    location=[airport_lat, airport_lon],
    popup='Closest Airport: KTIX',
    icon=folium.Icon(color='purple', icon='plane', prefix='fa')
).add_to(site_map)

# --- Draw Line Between Launch Site and Airport ---
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [airport_lat, airport_lon]],
    color='purple',
    weight=2,
    tooltip='Distance to Airport'
).add_to(site_map)

# --- Add Distance Label ---
folium.Marker(
    location=[(launch_site_lat + airport_lat)/2, (launch_site_lon + airport_lon)/2],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#6c3483;"><b>{:.2f} KM</b></div>'.format(distance_km),
    )
).add_to(site_map)

# --- Display Map ---
site_map


In [33]:
import folium
from folium.features import DivIcon
from math import radians, cos, sin, asin, sqrt

# --- Launch Site Coordinates (e.g., KSC LC-39A) ---
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# --- Closest Seaport Coordinates (e.g., Port Canaveral) ---
seaport_lat = 28.4132
seaport_lon = -80.6095

# --- Haversine Distance Function ---
def calculate_distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Earth radius in kilometers
    return c * r

# --- Calculate Distance ---
distance_km = calculate_distance(launch_site_lat, launch_site_lon, seaport_lat, seaport_lon)

# --- Create Map Centered on Launch Site ---
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# --- Add Launch Site Marker ---
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup='Launch Site',
    icon=folium.Icon(color='green')
).add_to(site_map)

# --- Add Seaport Marker ---
folium.Marker(
    location=[seaport_lat, seaport_lon],
    popup='Closest Seaport: Port Canaveral',
    icon=folium.Icon(color='darkblue', icon='ship', prefix='fa')
).add_to(site_map)

# --- Draw Line Between Launch Site and Seaport ---
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [seaport_lat, seaport_lon]],
    color='navy',
    weight=2,
    tooltip='Distance to Seaport'
).add_to(site_map)

# --- Add Distance Label ---
folium.Marker(
    location=[(launch_site_lat + seaport_lat)/2, (launch_site_lon + seaport_lon)/2],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#154360;"><b>{:.2f} KM</b></div>'.format(distance_km),
    )
).add_to(site_map)

# --- Display Map ---
site_map


In [26]:
import folium
from folium.plugins import MousePosition
from folium.features import DivIcon
from math import radians, cos, sin, asin, sqrt

# --- Coordinates ---
# Launch site (example: KSC LC-39A)
launch_site_lat = 28.573255
launch_site_lon = -80.646895

# Feature coordinates (e.g., coastline, city, railway, highway)
# Use MousePosition to hover and record these manually
feature_lat = 28.56367     # Example: coastline
feature_lon = -80.57163

# --- Distance Calculation ---
def calculate_distance(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Earth radius in kilometers
    return c * r

distance_km = calculate_distance(launch_site_lat, launch_site_lon, feature_lat, feature_lon)

# --- Map Setup ---
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=10)

# Add MousePosition plugin for manual coordinate selection
mouse_position = MousePosition(
    position='topright',
    separator=' | ',
    prefix='Feature Coordinates:',
    lat_formatter="function(num) {return L.Util.formatNum(num, 6);}",
    lng_formatter="function(num) {return L.Util.formatNum(num, 6);}"
)
site_map.add_child(mouse_position)

# --- Markers ---
# Launch site marker
folium.Marker(
    location=[launch_site_lat, launch_site_lon],
    popup='Launch Site',
    icon=folium.Icon(color='green')
).add_to(site_map)

# Feature marker (e.g., coastline, city, etc.)
folium.Marker(
    location=[feature_lat, feature_lon],
    popup='Closest Feature',
    icon=folium.Icon(color='blue')
).add_to(site_map)

# --- Line Between Launch Site and Feature ---
coordinates = [[launch_site_lat, launch_site_lon], [feature_lat, feature_lon]]
lines = folium.PolyLine(locations=coordinates, weight=2, color='purple', tooltip='Distance Line')
site_map.add_child(lines)

# --- Distance Label ---
folium.Marker(
    location=[(launch_site_lat + feature_lat)/2, (launch_site_lon + feature_lon)/2],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance_km),
    )
).add_to(site_map)

# --- Display Map ---
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
